# Advanced Python

1. Iterator
   1. List comprehension
   2. Generator
   3. Generator expression
2. Stack frame and import mechanism
3. Descriptor and meta class
4. Type introspection and abstract base class (abc)

In [ ]:
import pprint

# Iterator

While processing data in memory, they are iterated one by one.  Assume we have 10 elements in a list.

In [ ]:
data = list(range(10))
print(data, type(data))

Python uses the [iterator protocol](https://docs.python.org/3/library/stdtypes.html#iterator-types) to get one element a time:

In [ ]:
class ListIterator:

    def __init__(self, data):
        self.data = data
        self.it = None

    def __iter__(self):
        return self

    def __next__(self):
        if None is self.it:
            self.it = 0
        elif self.it >= len(self.data)-1:
            raise StopIteration
        else:
            self.it += 1
        return self.data[self.it]

The `for ... in ...` construct applies to the iterator object.  Every time the construct needs the next element, `ListIterator.__next__()` is called:

In [ ]:
list_iterator = ListIterator(data)
print(list_iterator)
print(dir(list_iterator))
for i in list_iterator:
    print(i)

Of course, you don't really need to write your own `ListIterator` for iterating a list, because Python builds in an iterator already:

In [ ]:
list_iterator2 = iter(data)
print(list_iterator2)
print(dir(list_iterator2))
for i in list_iterator2:
    print(i)

The built-in iterator is created by calling the `__iter__()` method on the container object (`iter()` simply does it for you):

In [ ]:
list_iterator3 = data.__iter__()
print(list_iterator3)
for i in list_iterator3:
    print(i)

And the `for ... in ...` construct actually knows about the iterator protocol:

In [ ]:
for i in data:
    print(i)

## List comprehension

List comprehension is the construct `[... for ... in ...]`.  Python borrowed the syntax of list comprehension from other languages, e.g., Haskell, and it follows the iterator protocol.  It is very convenient.  For example, the above `for` loop can be replaced by a one-liner:

In [ ]:
print("\n".join([str(i) for i in data]))

## Generator

In [ ]:
def list_generator(input_data):
    for i in input_data:
        yield i

generator = list_generator(data)
print(generator)
print(dir(generator))
for i in list_generator(data):
    print(i)

## Generator expression

A more convenient way of creating a generator is to use the generator expression `(... for ... in ...)`.  Note this looks like the list comprehension `[... for ... in ...]`, but uses parentheses to replace the brackets.

In [ ]:
generator2 = (i for i in data)
print(generator2)
print(dir(generator2))
for i in generator2:
    print(i)

By using the generator expression, the data printing one-liner can drop the brackets:

In [ ]:
print("\n".join(str(i) for i in data))
# Compare the the list comprehension:
# print("\n".join( [ str(i) for i in data ] ))